In [1]:
import aqt

from anki import scheduler_pb2

from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns
import scipy

import datetime, itertools, time

In [2]:
browser = aqt.dialogs._dialogs['Browser'][1]
table = browser.table
model = table._model

In [47]:
reviews = [model._state.get_review(item) for item in model._items]
cards = [model._state.get_card(item) for item in model._items]
notes = [model._state.get_note(item) for item in model._items]

In [4]:
t = time.time()

In [7]:
t

1673460776.3114223

In [47]:
d = datetime.datetime.fromtimestamp(t)

In [48]:
d

datetime.datetime(2023, 1, 9, 16, 17, 38, 571518)

In [49]:
d.timestamp()

1673306258.571518

In [41]:
int(t*1000)

1673306258571

In [15]:
CardAnswer = scheduler_pb2.CardAnswer

In [12]:
c0 = cards[0]
c0.__dict__

{'col': <anki.collection.Collection at 0x7f1b180e61d0>,
 'timer_started': None,
 '_render_output': None,
 'id': 1659061121013,
 '_note': None,
 'nid': 1659061121012,
 'did': 1659060767065,
 'ord': 0,
 'mod': 1673403815,
 'usn': 0,
 'type': 2,
 'queue': 2,
 'due': 36500,
 'ivl': 36500,
 'factor': 3750,
 'reps': 17,
 'lapses': 2,
 'left': 0,
 'odue': 0,
 'odid': 0,
 'flags': 0,
 'original_position': 1,
 'custom_data': 'blah'}

In [88]:
c0.load()
c0.__dict__

{'col': <anki.collection.Collection at 0x7f9a9409b180>,
 'timer_started': None,
 '_render_output': None,
 'id': 1659061121013,
 '_note': None,
 'nid': 1659061121012,
 'did': 1659060767065,
 'ord': 0,
 'mod': 1673307782,
 'usn': -1,
 'type': 3,
 'queue': 1,
 'due': 1673308479,
 'ivl': 1,
 'factor': 2300,
 'reps': 2,
 'lapses': 1,
 'left': 1,
 'odue': 0,
 'odid': 0,
 'flags': 0,
 'original_position': 1,
 'custom_data': 'blah'}

In [33]:
c0_states = model.col._backend.get_scheduling_states(c0.id)
c0_states

current {
  normal {
    review {
      scheduled_days: 3
      elapsed_days: 162
      ease_factor: 2.5
    }
  }
}
again {
  normal {
    relearning {
      review {
        scheduled_days: 1
        ease_factor: 2.3
        lapses: 1
      }
      learning {
        remaining_steps: 1
        scheduled_secs: 600
      }
    }
  }
}
hard {
  normal {
    review {
      scheduled_days: 5
      ease_factor: 2.35
    }
  }
}
good {
  normal {
    review {
      scheduled_days: 212
      ease_factor: 2.5
    }
  }
}
easy {
  normal {
    review {
      scheduled_days: 540
      ease_factor: 2.65
    }
  }
}

In [4]:
def build_answer(card, rating, seconds_taken, answered_at = None):
    if answered_at is None:
        answered_at = datetime.datetime.now()
    if type(answered_at) == datetime.datetime:
        answered_at = int(d.timestamp() * 1000)
    ms_taken = int(seconds_taken * 1000)
    states = model.col._backend.get_scheduling_states(card.id)
    if rating == CardAnswer.AGAIN:
        new_state = states.again
    elif rating == CardAnswer.HARD:
        new_state = states.hard
    elif rating == CardAnswer.GOOD:
        new_state = states.good
    elif rating == CardAnswer.EASY:
        new_state = states.easy
    else:
        raise Exception("invalid rating")
        
    answer = CardAnswer(
        card_id=card.id,
        current_state=states.current,
        new_state=states.again,
        rating=rating,
        answered_at_millis=answered_at,
        milliseconds_taken=ms_taken,
    )
    print(f"states: {states}")
    print(f"new_state: {new_state}")
    return answer

In [5]:
def answer_card(answer):
    model.col._backend.answer_card_raw(answer.SerializeToString())

1673411488.387 POST /_anki/cardStats


In [75]:
answer = build_answer(c0, CardAnswer.AGAIN, 3600)

In [77]:
answer

card_id: 1659061121013
current_state {
  normal {
    review {
      scheduled_days: 3
      elapsed_days: 162
      ease_factor: 2.5
    }
  }
}
new_state {
  normal {
    relearning {
      review {
        scheduled_days: 1
        ease_factor: 2.3
        lapses: 1
      }
      learning {
        remaining_steps: 1
        scheduled_secs: 600
      }
    }
  }
}
answered_at_millis: 1673306258571
milliseconds_taken: 3600000

In [83]:
answer_card(answer)

In [90]:
d = datetime.datetime(2022, 12, 31)
d

datetime.datetime(2022, 12, 31, 0, 0)

In [91]:
answer = build_answer(c0, CardAnswer.AGAIN, 3600, answered_at = d)
answer

card_id: 1659061121013
current_state {
  normal {
    relearning {
      review {
        scheduled_days: 1
        elapsed_days: 1
        ease_factor: 2.3
        lapses: 1
      }
      learning {
        remaining_steps: 1
        scheduled_secs: 600
      }
    }
  }
}
new_state {
  normal {
    relearning {
      review {
        scheduled_days: 1
        ease_factor: 2.3
        lapses: 1
      }
      learning {
        remaining_steps: 1
        scheduled_secs: 600
      }
    }
  }
}
answered_at_millis: 1672470000000
milliseconds_taken: 3600000

In [92]:
answer_card(answer)

In [93]:
c0.load()
c0.__dict__

{'col': <anki.collection.Collection at 0x7f9a9409b180>,
 'timer_started': None,
 '_render_output': None,
 'id': 1659061121013,
 '_note': None,
 'nid': 1659061121012,
 'did': 1659060767065,
 'ord': 0,
 'mod': 1673308105,
 'usn': -1,
 'type': 3,
 'queue': 1,
 'due': 1673308828,
 'ivl': 1,
 'factor': 2300,
 'reps': 3,
 'lapses': 1,
 'left': 1,
 'odue': 0,
 'odid': 0,
 'flags': 0,
 'original_position': 1,
 'custom_data': 'blah'}

In [95]:
d = datetime.datetime(2022, 12, 31, 0, 1)
answer = build_answer(c0, CardAnswer.GOOD, 3600, answered_at = d)
answer_card(answer)

In [96]:
c0.load()
c0.__dict__

{'col': <anki.collection.Collection at 0x7f9a9409b180>,
 'timer_started': None,
 '_render_output': None,
 'id': 1659061121013,
 '_note': None,
 'nid': 1659061121012,
 'did': 1659060767065,
 'ord': 0,
 'mod': 1673308247,
 'usn': -1,
 'type': 3,
 'queue': 1,
 'due': 1673308856,
 'ivl': 1,
 'factor': 2300,
 'reps': 4,
 'lapses': 1,
 'left': 1,
 'odue': 0,
 'odid': 0,
 'flags': 0,
 'original_position': 1,
 'custom_data': 'blah'}

In [14]:
d = datetime.datetime(2022, 12, 31, 0, 7)
answer = build_answer(c0, CardAnswer.AGAIN, 5.5, answered_at = d)
answer_card(answer)
c0.load()
c0.__dict__

states: current {
  normal {
    relearning {
      review {
        scheduled_days: 1
        elapsed_days: 1
        ease_factor: 3.55
        lapses: 3
      }
      learning {
        remaining_steps: 1
        scheduled_secs: 600
      }
    }
  }
}
again {
  normal {
    relearning {
      review {
        scheduled_days: 1
        ease_factor: 3.55
        lapses: 3
      }
      learning {
        remaining_steps: 1
        scheduled_secs: 600
      }
    }
  }
}
hard {
  normal {
    relearning {
      review {
        scheduled_days: 1
        ease_factor: 3.55
        lapses: 3
      }
      learning {
        remaining_steps: 1
        scheduled_secs: 900
      }
    }
  }
}
good {
  normal {
    review {
      scheduled_days: 1
      elapsed_days: 1
      ease_factor: 3.55
      lapses: 3
    }
  }
}
easy {
  normal {
    review {
      scheduled_days: 2
      ease_factor: 3.55
      lapses: 3
    }
  }
}

new_state: normal {
  relearning {
    review {
      scheduled_day

{'col': <anki.collection.Collection at 0x7f1b180e61d0>,
 'timer_started': None,
 '_render_output': None,
 'id': 1659061121013,
 '_note': None,
 'nid': 1659061121012,
 'did': 1659060767065,
 'ord': 0,
 'mod': 1673403982,
 'usn': -1,
 'type': 3,
 'queue': 1,
 'due': 1673404629,
 'ivl': 1,
 'factor': 3550,
 'reps': 19,
 'lapses': 3,
 'left': 1,
 'odue': 0,
 'odid': 0,
 'flags': 0,
 'original_position': 1,
 'custom_data': 'blah'}

1673403988.585 GET /_anki/pages/card-info.html
1673403988.601 GET /_anki/pages/card-info.css
1673403988.602 GET /_anki/pages/card-info.js
1673403988.828 POST /_anki/i18nResources
1673403988.870 POST /_anki/cardStats


In [16]:
d = datetime.datetime(2023, 1, 9, 16, 57)
answer = build_answer(c0, CardAnswer.AGAIN, 5.5, answered_at = d)
answer_card(answer)
c0.load()
c0.__dict__

states: current {
  normal {
    relearning {
      review {
        scheduled_days: 1
        elapsed_days: 1
        ease_factor: 2.65
        lapses: 6
      }
      learning {
        remaining_steps: 1
        scheduled_secs: 600
      }
    }
  }
}
again {
  normal {
    relearning {
      review {
        scheduled_days: 1
        ease_factor: 2.65
        lapses: 6
      }
      learning {
        remaining_steps: 1
        scheduled_secs: 600
      }
    }
  }
}
hard {
  normal {
    relearning {
      review {
        scheduled_days: 1
        ease_factor: 2.65
        lapses: 6
      }
      learning {
        remaining_steps: 1
        scheduled_secs: 900
      }
    }
  }
}
good {
  normal {
    review {
      scheduled_days: 1
      elapsed_days: 1
      ease_factor: 2.65
      lapses: 6
    }
  }
}
easy {
  normal {
    review {
      scheduled_days: 2
      ease_factor: 2.65
      lapses: 6
    }
  }
}

new_state: normal {
  relearning {
    review {
      scheduled_day

{'col': <anki.collection.Collection at 0x7f47f85524a0>,
 'timer_started': None,
 '_render_output': None,
 'id': 1659061121013,
 '_note': None,
 'nid': 1659061121012,
 'did': 1659060767065,
 'ord': 0,
 'mod': 1673411585,
 'usn': -1,
 'type': 3,
 'queue': 1,
 'due': 1673412301,
 'ivl': 1,
 'factor': 2650,
 'reps': 26,
 'lapses': 6,
 'left': 1,
 'odue': 0,
 'odid': 0,
 'flags': 0,
 'original_position': 1,
 'custom_data': 'blah'}

1673411593.540 GET /_anki/pages/card-info.html
1673411593.555 GET /_anki/pages/card-info.css
1673411593.557 GET /_anki/pages/card-info.js
1673411593.692 POST /_anki/i18nResources
1673411593.812 POST /_anki/cardStats


Traceback (most recent call last):
  File "/mnt/Work/Repos/ankitects/anki-221201-1423/qt/aqt/webview.py", line 562, in handler
    cb(val)
  File "/mnt/Work/Repos/ankitects/anki-221201-1423/qt/aqt/editor.py", line 701, in <lambda>
    self.web.evalWithCallback("saveNow(%d)" % keepFocus, lambda res: callback())
  File "/mnt/Work/Repos/ankitects/anki-221201-1423/qt/aqt/utils.py", line 1188, in <lambda>
    self.editor.call_after_note_saved(lambda: func(self, *args, **kwargs))
TypeError: replay_card_histories() takes 1 positional argument but 2 were given


1673411640.044 GET /_anki/pages/card-info.html
1673411640.054 GET /_anki/pages/card-info.css
1673411640.055 GET /_anki/pages/card-info.js
1673411640.299 POST /_anki/i18nResources
1673411640.311 POST /_anki/cardStats
1673411891.224 GET /_anki/legacyPageData
1673411891.225 GET /_anki/legacyPageData
1673411891.240 GET /_anki/css/webview.css
1673411891.241 GET /_anki/css/deckbrowser.css
1673411891.242 GET /_anki/js/webview.js
1673411891.243 GET /_anki/js/vendor/jquery.min.js
1673411891.247 GET /_anki/js/vendor/jquery-ui.min.js
1673411891.248 GET /_anki/imgs/gears.svg
1673411891.249 GET /_anki/js/deckbrowser.js
1673411891.252 GET /_anki/css/webview.css
1673411891.254 GET /_anki/css/toolbar.css
1673411891.256 GET /_anki/css/toolbar-bottom.css
1673411891.257 GET /_anki/js/webview.js
1673411891.258 GET /_anki/js/vendor/jquery.min.js


In [111]:
d = datetime.datetime(2023, 1, 9, 16, 18)
answer = build_answer(c0, CardAnswer.EASY, 5.5, answered_at = d)
answer_card(answer)
c0.load()
c0.__dict__

states: current {
  filtered {
    rescheduling {
      original_state {
        relearning {
          review {
            scheduled_days: 1
            elapsed_days: 1
            ease_factor: 2.1
            lapses: 2
          }
          learning {
            remaining_steps: 1
            scheduled_secs: 600
          }
        }
      }
    }
  }
}
again {
  filtered {
    rescheduling {
      original_state {
        relearning {
          review {
            scheduled_days: 1
            ease_factor: 2.1
            lapses: 2
          }
          learning {
            remaining_steps: 1
            scheduled_secs: 600
          }
        }
      }
    }
  }
}
hard {
  filtered {
    rescheduling {
      original_state {
        relearning {
          review {
            scheduled_days: 1
            ease_factor: 2.1
            lapses: 2
          }
          learning {
            remaining_steps: 1
            scheduled_secs: 900
          }
        }
      }
    }
  }

{'col': <anki.collection.Collection at 0x7f9a9409b180>,
 'timer_started': None,
 '_render_output': None,
 'id': 1659061121013,
 '_note': None,
 'nid': 1659061121012,
 'did': 1673308789192,
 'ord': 0,
 'mod': 1673309323,
 'usn': -1,
 'type': 3,
 'queue': 1,
 'due': 1673309967,
 'ivl': 1,
 'factor': 2100,
 'reps': 13,
 'lapses': 2,
 'left': 1,
 'odue': 0,
 'odid': 1659060767065,
 'flags': 0,
 'original_position': 1,
 'custom_data': 'blah'}

1673309332.497 GET /_anki/pages/card-info.html
1673309332.509 GET /_anki/pages/card-info.css
1673309332.510 GET /_anki/pages/card-info.js
1673309332.604 POST /_anki/i18nResources
1673309332.682 POST /_anki/cardStats
1673309343.800 GET /_anki/legacyPageData
1673309343.801 GET /_anki/legacyPageData
1673309343.832 GET /_anki/css/webview.css
1673309343.833 GET /_anki/css/reviewer.css
1673309343.840 GET /_anki/css/webview.css
1673309343.841 GET /_anki/css/toolbar-bottom.css
1673309343.843 GET /_anki/css/reviewer-bottom.css
1673309343.843 GET /_anki/js/webview.js
1673309343.847 GET /_anki/js/vendor/jquery.min.js
1673309343.847 GET /_anki/js/reviewer-bottom.js
1673309343.853 GET /_anki/js/webview.js
1673309343.855 GET /_anki/js/mathjax.js
1673309343.855 GET /_anki/js/reviewer.js
1673309343.857 GET /_anki/js/vendor/mathjax/tex-chtml.js
1673309343.983 GET /_anki/js/vendor/mathjax/input/tex/extensions/noerrors.js
1673309343.984 GET /_anki/js/vendor/mathjax/input/tex/extensions/mathtools.js
16733

In [104]:
d = datetime.datetime.now()
answer = build_answer(c0, CardAnswer.EASY, 5, answered_at = d)
answer_card(answer)
c0.load()
c0.__dict__

states: current {
  normal {
    relearning {
      review {
        scheduled_days: 1
        elapsed_days: 1
        ease_factor: 2.3
        lapses: 1
      }
      learning {
        remaining_steps: 1
        scheduled_secs: 600
      }
    }
  }
}
again {
  normal {
    relearning {
      review {
        scheduled_days: 1
        ease_factor: 2.3
        lapses: 1
      }
      learning {
        remaining_steps: 1
        scheduled_secs: 600
      }
    }
  }
}
hard {
  normal {
    relearning {
      review {
        scheduled_days: 1
        ease_factor: 2.3
        lapses: 1
      }
      learning {
        remaining_steps: 1
        scheduled_secs: 900
      }
    }
  }
}
good {
  normal {
    review {
      scheduled_days: 1
      elapsed_days: 1
      ease_factor: 2.3
      lapses: 1
    }
  }
}
easy {
  normal {
    review {
      scheduled_days: 2
      ease_factor: 2.3
      lapses: 1
    }
  }
}

new_state: normal {
  review {
    scheduled_days: 2
    ease_factor: 2.

{'col': <anki.collection.Collection at 0x7f9a9409b180>,
 'timer_started': None,
 '_render_output': None,
 'id': 1659061121013,
 '_note': None,
 'nid': 1659061121012,
 'did': 1659060767065,
 'ord': 0,
 'mod': 1673308619,
 'usn': -1,
 'type': 3,
 'queue': 1,
 'due': 1673309233,
 'ivl': 1,
 'factor': 2300,
 'reps': 8,
 'lapses': 1,
 'left': 1,
 'odue': 0,
 'odid': 0,
 'flags': 0,
 'original_position': 1,
 'custom_data': 'blah'}

1673308662.205 POST /_anki/cardStats
1673308663.347 POST /_anki/cardStats
1673308671.798 GET /_anki/pages/card-info.html
1673308671.920 GET /_anki/pages/card-info.css
1673308671.921 GET /_anki/pages/card-info.js
1673308672.002 POST /_anki/i18nResources
1673308672.022 POST /_anki/cardStats
1673308696.199 POST /_anki/cardStats
1673308717.248 GET /_anki/legacyPageData
1673308717.249 GET /_anki/legacyPageData
1673308717.262 GET /_anki/css/webview.css
1673308717.263 GET /_anki/css/overview.css
1673308717.263 GET /_anki/js/webview.js
1673308717.264 GET /_anki/js/vendor/jquery.min.js
1673308717.274 GET /_anki/css/webview.css
1673308717.274 GET /_anki/css/toolbar.css
1673308717.276 GET /_anki/css/toolbar-bottom.css
1673308717.278 GET /_anki/js/webview.js
1673308717.279 GET /_anki/js/vendor/jquery.min.js
1673308718.793 GET /_anki/legacyPageData
1673308718.794 GET /_anki/legacyPageData
1673308718.810 GET /_anki/css/webview.css
1673308718.811 GET /_anki/css/reviewer.css
1673308718.812 GET /_anki/

In [100]:
answer

card_id: 1659061121013
current_state {
  normal {
    relearning {
      review {
        scheduled_days: 1
        elapsed_days: 1
        ease_factor: 2.3
        lapses: 1
      }
      learning {
        remaining_steps: 1
        scheduled_secs: 600
      }
    }
  }
}
new_state {
  normal {
    relearning {
      review {
        scheduled_days: 1
        ease_factor: 2.3
        lapses: 1
      }
      learning {
        remaining_steps: 1
        scheduled_secs: 600
      }
    }
  }
}
rating: EASY
answered_at_millis: 1673308448151
milliseconds_taken: 3600000

In [8]:
cards[0].__dict__

{'col': <anki.collection.Collection at 0x7f9a9409b180>,
 'timer_started': None,
 '_render_output': None,
 'id': 1659061121013,
 '_note': None,
 'nid': 1659061121012,
 'did': 1659060767065,
 'ord': 0,
 'mod': 1659295892,
 'usn': 2002,
 'type': 2,
 'queue': 2,
 'due': 1714,
 'ivl': 3,
 'factor': 2500,
 'reps': 1,
 'lapses': 0,
 'left': 2002,
 'odue': 0,
 'odid': 0,
 'flags': 0,
 'original_position': 1,
 'custom_data': 'blah'}

In [9]:
queued_cards = model.col.sched.get_queued_cards()

In [10]:
queued_cards

cards {
  card {
    id: 1659407673731
    note_id: 1659407673731
    deck_id: 1659060767065
    mtime_secs: 1659408224
    usn: 1745
    ctype: 1
    queue: 1
    due: 1659408263
    reps: 1
    remaining_steps: 2002
  }
  queue: LEARNING
  states {
    current {
      normal {
        learning {
          remaining_steps: 2
          scheduled_secs: 60
        }
      }
    }
    again {
      normal {
        learning {
          remaining_steps: 2
          scheduled_secs: 60
        }
      }
    }
    hard {
      normal {
        learning {
          remaining_steps: 2
          scheduled_secs: 330
        }
      }
    }
    good {
      normal {
        learning {
          remaining_steps: 1
          scheduled_secs: 600
        }
      }
    }
    easy {
      normal {
        review {
          scheduled_days: 2
          ease_factor: 2.5
        }
      }
    }
  }
}
new_count: 24
learning_count: 12
review_count: 24

In [16]:
queued_cards.cards[0].card.note_id

1659407673731

In [18]:
model.col.get_note(queued_cards.cards[0].card.note_id).__dict__

{'col': <anki.collection.Collection at 0x7f9a9409b180>,
 'id': 1659407673731,
 'guid': 'n_Tfp.)F?[',
 'mid': 1539794822805,
 'mod': 1672951084,
 'usn': 1963,
 'tags': ['CET-07.04-IntegrationOfRationalFunctionsByPartialFractions'],
 'fields': ['CET-07.04-Xm04',
  '',
  '220801-2032: trouble:&nbsp;<span style="color: var(--field-fg); font-family: -apple-system-body; background: var(--field-bg);">arithmetic: inverted a fraction.</span>'],
 '_fmap': {'Front': (0,
   {'name': 'Front',
    'ord': 0,
    'sticky': False,
    'rtl': False,
    'font': 'Liberation Sans',
    'size': 20,
    'description': '',
    'plainText': False,
    'collapsed': False,
    'media': []}),
  'Back': (1,
   {'name': 'Back',
    'ord': 1,
    'sticky': False,
    'rtl': False,
    'font': 'Arial',
    'size': 20,
    'description': '',
    'plainText': False,
    'collapsed': False,
    'media': []}),
  'Annotations': (2,
   {'name': 'Annotations',
    'ord': 2,
    'sticky': False,
    'rtl': False,
    'font'

In [30]:
model.col.get_note(queued_cards.cards[0].card.note_id).__dict__

In [5]:
cards[0].__dict__

{'col': <anki.collection.Collection at 0x7f8c20228f40>,
 'timer_started': None,
 '_render_output': None,
 'id': 1659061121013,
 '_note': None,
 'nid': 1659061121012,
 'did': 1659060767065,
 'ord': 0,
 'mod': 1673309406,
 'usn': 2005,
 'type': 2,
 'queue': 2,
 'due': 1875,
 'ivl': 2,
 'factor': 2550,
 'reps': 17,
 'lapses': 2,
 'left': 0,
 'odue': 0,
 'odid': 0,
 'flags': 0,
 'original_position': 1,
 'custom_data': 'blah'}

In [ ]:
model.col.get

```sql
CREATE TABLE col (
    id              integer primary key,
    crt             integer not null,
    mod             integer not null,
    scm             integer not null,
    ver             integer not null,
    dty             integer not null,
    usn             integer not null,
    ls              integer not null,
    conf            text not null,
    models          text not null,
    decks           text not null,
    dconf           text not null,
    tags            text not null,
    semver           TEXT,
    extended_version INTEGER
);

CREATE TABLE notes (
    id              integer primary key,
    guid            text not null,
    mid             integer not null,
    mod             integer not null,
    usn             integer not null,
    tags            text not null,
    flds            text not null,
    sfld            integer not null,
    csum            integer not null,
    flags           integer not null,
    data            text not null
);

CREATE TABLE cards (
    id              integer primary key,
    nid             integer not null,
    did             integer not null,
    ord             integer not null,
    mod             integer not null,
    usn             integer not null,
    type            integer not null,
    queue           integer not null,
    due             integer not null,
    ivl             integer not null,
    factor          integer not null,
    reps            integer not null,
    lapses          integer not null,
    left            integer not null,
    odue            integer not null,
    odid            integer not null,
    flags           integer not null,
    data            text not null
);

CREATE TABLE sqlite_stat1(tbl,idx,stat);

CREATE TABLE sqlite_stat4(tbl,idx,neq,nlt,ndlt,sample);

CREATE TABLE revlog (
    id              integer primary key,
    cid             integer not null,
    usn             integer not null,
    ease            integer not null,
    ivl             integer not null,
    lastIvl         integer not null,
    factor          integer not null,
    time            integer not null,
    type            integer not null,
    feedback TEXT DEFAULT "" NOT NULL,
    tags TEXT DEFAULT "" NOT NULL,
    mod INTEGER default 0 not null
);

CREATE TABLE deck_config (
    id integer PRIMARY KEY NOT NULL,
    name text NOT NULL COLLATE unicase,
    mtime_secs integer NOT NULL,
    usn integer NOT NULL,
    config blob NOT NULL
);

CREATE TABLE config (
    KEY text NOT NULL PRIMARY KEY,
    usn integer NOT NULL,
    mtime_secs integer NOT NULL,
    val blob NOT NULL
) without rowid;

CREATE TABLE fields (
    ntid integer NOT NULL,
    ord integer NOT NULL,
    name text NOT NULL COLLATE unicase,
    config blob NOT NULL,
    PRIMARY KEY (ntid, ord)
) without rowid;

CREATE TABLE templates (
    ntid integer NOT NULL,
    ord integer NOT NULL,
    name text NOT NULL COLLATE unicase,
    mtime_secs integer NOT NULL,
    usn integer NOT NULL,
    config blob NOT NULL,
    PRIMARY KEY (ntid, ord)
) without rowid;

CREATE TABLE notetypes (
    id integer NOT NULL PRIMARY KEY,
    name text NOT NULL COLLATE unicase,
    mtime_secs integer NOT NULL,
    usn integer NOT NULL,
    config blob NOT NULL
);

CREATE TABLE decks (
    id integer PRIMARY KEY NOT NULL,
    name text NOT NULL COLLATE unicase,
    mtime_secs integer NOT NULL,
    usn integer NOT NULL,
    common blob NOT NULL,
    kind blob NOT NULL
);

CREATE TABLE tags (
    tag text NOT NULL PRIMARY KEY COLLATE unicase,
    usn integer NOT NULL,
    collapsed boolean NOT NULL,
    config blob NULL
) without rowid;

CREATE TABLE graves (
    oid integer NOT NULL,
    type integer NOT NULL,
    usn integer NOT NULL,
    PRIMARY KEY (oid, type)
) WITHOUT ROWID;

CREATE INDEX ix_notes_usn on notes (usn);
CREATE INDEX ix_cards_usn on cards (usn);
CREATE INDEX ix_cards_nid on cards (nid);
CREATE INDEX ix_cards_sched on cards (did, queue, due);
CREATE INDEX ix_notes_csum on notes (csum);
CREATE UNIQUE INDEX idx_fields_name_ntid ON fields (name, ntid);
CREATE UNIQUE INDEX idx_templates_name_ntid ON templates (name, ntid);
CREATE INDEX idx_templates_usn ON templates (usn);
CREATE UNIQUE INDEX idx_notetypes_name ON notetypes (name);
CREATE INDEX idx_notetypes_usn ON notetypes (usn);
CREATE UNIQUE INDEX idx_decks_name ON decks (name);
CREATE INDEX idx_notes_mid ON notes (mid);
CREATE INDEX idx_cards_odid ON cards (odid) WHERE odid != 0;
CREATE INDEX idx_graves_pending ON graves (usn);
```

In [7]:
reviews = [model._state.get_review(item) for item in model._items]
cards = [model._state.get_card(item) for item in model._items]
notes = [model._state.get_note(item) for item in model._items]

In [6]:
model.col._backend.replay_card_histories(table.get_selected_card_ids())

card: true
mtime: true
browser_table: true
study_queues: true

In [14]:
table.get_single_selected_card().__dict__

{'col': <anki.collection.Collection at 0x7f5e2c042540>,
 'timer_started': None,
 '_render_output': None,
 'id': 1659061207486,
 '_note': <anki.notes.Note at 0x7f5cc17423a0>,
 'nid': 1659061207485,
 'did': 1659060767065,
 'ord': 0,
 'mod': 1673460783,
 'usn': 0,
 'type': 2,
 'queue': 2,
 'due': 2,
 'ivl': 2,
 'factor': 2500,
 'reps': 1,
 'lapses': 0,
 'left': 0,
 'odue': 0,
 'odid': 0,
 'flags': 0,
 'original_position': None,
 'custom_data': ''}